In [3]:
#use ipykernal
#!pip install mediapipe

In [4]:
import mediapipe as mp
import cv2

# Gesture Recognition class
class GestureRecognition:
    def __init__(self):
        self.hands = mp.solutions.hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.5)
        self.current_gesture = None

    def detect_gesture(self, image):
        results = self.hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.multi_hand_landmarks:
            hand_landmarks = results.multi_hand_landmarks[0]
            self.current_gesture = self.get_gesture(hand_landmarks)
            # Draw landmarks on the image for visualization (optional)
            mp.solutions.drawing_utils.draw_landmarks(image, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)

    def get_gesture(self, hand_landmarks):
        # Get fingertip positions
        thumb_tip = hand_landmarks.landmark[4]
        index_finger_tip = hand_landmarks.landmark[8]
        middle_finger_tip = hand_landmarks.landmark[12]
        ring_finger_tip = hand_landmarks.landmark[16]
        little_finger_tip = hand_landmarks.landmark[20]

        # Define tips and bases for fingers
        tips = [8, 12, 16, 20]  # Index, Middle, Ring, Pinky tips
        bases = [5, 9, 13, 17]  # Respective knuckles

        # Check for "Open Palm" gesture
        if all(hand_landmarks.landmark[tip].y < hand_landmarks.landmark[base].y for tip, base in zip(tips, bases)):
            return "Hello"

        # Check for "Okay" gesture
        if thumb_tip.y < index_finger_tip.y < middle_finger_tip.y < ring_finger_tip.y < little_finger_tip.y:
            return "Okay"

        # Check for "Dislike" gesture
        elif thumb_tip.y > index_finger_tip.y > middle_finger_tip.y > ring_finger_tip.y > little_finger_tip.y:
            return "Dislike"
        

        # Check for "Point" gesture (index finger extended and pointing)
        elif index_finger_tip.y < middle_finger_tip.y and abs(index_finger_tip.x - middle_finger_tip.x) < 0.2:
            return "Point"

        # Check for "Victory" gesture (index and middle fingers up)
        elif index_finger_tip.y < thumb_tip.y and middle_finger_tip.y < thumb_tip.y:
            return "Victory"
        

        # Check for "Stop" gesture (at least 4 fingers extended)
        # We will check if the tip of each finger is above the knuckle (indicating it's extended)
        fingers = [thumb_tip, index_finger_tip, middle_finger_tip, ring_finger_tip, little_finger_tip]
        extended_fingers = sum([1 for finger in fingers if finger.y < hand_landmarks.landmark[0].y])  # count extended fingers
        if extended_fingers >= 4:
            return "Stop"

        # Check for "I Love You" gesture (index, little finger, and thumb extended)
        

        return None


# Initialize the gesture recognition class
gesture_recognition = GestureRecognition()

# Open the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect the gesture
    gesture_recognition.detect_gesture(frame)

    # Display the detected gesture on the frame
    gesture_text = gesture_recognition.current_gesture if gesture_recognition.current_gesture else "No gesture detected"
    cv2.putText(frame, f"Gesture: {gesture_text}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show the frame with the gesture output
    cv2.imshow('Gesture Recognition', frame)

    # Break the loop if the user presses the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close any open windows
cap.release()
cv2.destroyAllWindows()
